## GridWorld
Ph.D Leonarod A, Espinosa, M.Sc Andrej Scherbakov-Parland, BIT Kristoffer Kuvaja Adolfsson

### Bibliography:

* Sutton, Richard S., and Andrew G. Barto. Reinforcement learning: An introduction. MIT press, 2018.
http://incompleteideas.net/book/bookdraft2017nov5.pdf  (chapter 4)

In [1]:
# imports
import numpy as np

In [2]:
# Utilits
def printV(V, grid):
    for idx, row in enumerate(grid.grid):
        for idy, _ in enumerate(row):
            state = grid.m * idx + idy
            print('%.2f' % V[state], end='\t')
        print('\n')
    print('--------------------')

def printPolicy(policy, grid):
    for idx, row in enumerate(grid.grid):
        for idy, _ in enumerate(row):
            state = grid.m * idx + idy
            if state in grid.stateSpace:
                string = ''.join(policy[state])
                print(string, end='\t')
            else:
                print('', end='\t')
        print('\n')
    print('--------------------')

def printQ(Q, grid):
    for idx, row in enumerate(grid.grid):
        for idy, _ in enumerate(row):
            state = grid.m * idx + idy
            if state != grid.m * grid.n - 1:
                vals = [np.round(Q[state,action], 5) for action in grid.possibleActions]
                print(vals, end='\t')
        print('\n')
    print('--------------------')

def sampleReducedActionSpace(grid, action):
    actions = grid.possibleActions[:]
    actions.remove(action)
    sample = np.random.choice(actions)
    return sample

In [3]:
class WindyGrid(object):
    def __init__(self, m, n, wind):
        self.grid = np.zeros((m,n))                            # representation of the grid
        self.m = m
        self.n = n
        self.stateSpace = [i for i in range(self.m*self.n)]
        self.stateSpace.remove(28)                              # Terminal state
        self.stateSpacePlus = [i for i in range(self.m*self.n)] # State space + terminal state
        self.actionSpace = {'U': -self.m, 'D': self.m,
                            'L': -1, 'R': 1}
        self.possibleActions = ['U', 'D', 'L', 'R']
        self.agentPosition = 0
        self.wind = wind

    def isTerminalState(self, state):
        return state in self.stateSpacePlus and state not in self.stateSpace

    def getAgentRowAndColumn(self):                               # position of agent
        x = self.agentPosition // self.m
        y = self.agentPosition % self.n
        return x, y

    def setState(self, state):
        x, y = self.getAgentRowAndColumn()
        self.grid[x][y] = 0
        self.agentPosition = state
        x, y = self.getAgentRowAndColumn()
        self.grid[x][y] = 1

    def offGridMove(self, newState, oldState):
        # if we move into a row not in the grid
        if newState not in self.stateSpacePlus:
            return True
        # if we're trying to wrap around to next row
        elif oldState % self.m == 0 and newState  % self.m == self.m - 1:
            return True
        elif oldState % self.m == self.m - 1 and newState % self.m == 0:
            return True
        else:
            return False

    # Include wind stenght.
    def step(self, action):
        agentX, agentY = self.getAgentRowAndColumn()
        if agentX > 0:
            resultingState = self.agentPosition + self.actionSpace[action] + \
                            self.wind[agentY] * self.actionSpace['U']
            if resultingState < 0: #if the wind is trying to push agent off grid
                resultingState += self.m
        else:
            if action == 'L' or action == 'R':
                resultingState = self.agentPosition + self.actionSpace[action]
            else:
                resultingState = self.agentPosition + self.actionSpace[action] + \
                            self.wind[agentY] * self.actionSpace['U']
        #reward = -1 if not self.isTerminalState(resultingState) else 0
        reward = -1
        if not self.offGridMove(resultingState, self.agentPosition):
            self.setState(resultingState)
            return resultingState, reward, self.isTerminalState(resultingState), None
        else:
            return self.agentPosition, reward, self.isTerminalState(self.agentPosition), None

    def reset(self):
        self.agentPosition = 0
        self.grid = np.zeros((self.m,self.n))
        return self.agentPosition, False


    def render(self):
        print('------------------------------------------')
        for row in self.grid:
            for col in row:
                if col == 0:
                    print('-', end='\t')
                elif col == 1:
                    print('X', end='\t')
            print('\n')
        print('------------------------------------------')


## First visit Monte Carlo Prediction

In [4]:
def MC_first_visit(loop=500):

    grid = WindyGrid(6,6, wind=[0, 0, 1, 2, 1, 0])
    GAMMA = 1.0

    policy = {}                              #  a dictionary that maps each
    for state in grid.stateSpace:            #  state to the list of possible actions
        policy[state] = grid.possibleActions

    V = {}                                   # Initialize our initial estimate of the value
    for state in grid.stateSpacePlus:        # function. Each state gets a value of 0.
        V[state] = 0

    returns = {}                             #Initialize a dictionary that keeps a list
    for state in grid.stateSpace:            #of the returns for each state.
        returns[state] = []

    for i in range(loop):                     # Loop over 500 games,
        observation, done = grid.reset()     # resetting the grid and memory with each game.
        memory = []                          # empty list to keep track of the states visited
        statesReturns = []                   # and returns at each time step
        if i % 100 == 0:                     # Just to know if the game is running.
            print('starting episode', i)
        while not done:                      # While the game isn't done
            # attempt to follow the policy. In this case choose an action
            # according to the random equiprobable strategy.
            action = np.random.choice(policy[observation])
            observation_, reward, done, info = grid.step(action)  # Take that action, get new state, reward and done
            memory.append((observation, action, reward))
            observation = observation_

        # append terminal state
        memory.append((observation, action, reward))

        G = 0                                  # set G=0
        last = True                            # initialize a Boolean to keep track of the visit to the last state
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:                                    # Skip the terminal state and append the set of states
                statesReturns.append((state,G))      #  and returns to the statesReturns list.
            G = GAMMA*G + reward

        statesReturns.reverse()                  # to ge it in chronological order
        statesVisited = []                       # keep track of the visited states during the episode.
        for state, G in statesReturns:
            if state not in statesVisited:       # Iterate over the episode and see
                returns[state].append(G)         # if each state has been visited before.
                V[state] = np.mean(returns[state])
                statesVisited.append(state)

                #If it hasn't, meaning this is the agent's first visit, go ahead and append
                #the returns to the returns dictionary for that state.
                #Calculate the value function by taking the mean of the returns for that state, and finally,
                #append that state to the list of statesVisited.
    print("\n")
    printV(V, grid)

## Del 1:

- Använd *first visit* Monte Carlo Metoden

1. Öka vindstyrkan med en enhet.
    - Hur ändras slutvärdesfunktionen?
    - Svar: Det har blivit tydligt svårare omgivning att lösa eftersom värdena i genomsnitt över hela rutfältet har minskat (större negativt värde)

2. Hur ändras värdefunktion om man ändra gamma till:
    - 𝛾=0.5 Svar: Alla värden förutom några ändrades till -2.0. 
    - 𝛾=0,9 Svar: Flesta värden ändrar till -10 / -9.99, om inte någon dera av dessa så andra värden väldigt nära dessa.
    - 𝛾=0,95 Svar: Flesta värden ändrar till -9.99, om inte någon dera av dessa så andra värden väldigt nära dessa.


3. Testa rutnätsvärlden i storlekarna:
    - 8x8
        - Ändra på vinden, vad händer med värdefunktion?
            Svar: Med ökad vind och större rutnätverk ökar negativa värdena.
        - Prova med 𝛾=0,9, vad händer med värdefunktion?
            Svar:Liknande som tidigare flesta värden ändrar till -10 / -9.99 eller något liknande värde.
    - 10x10
        - Ändra på vinden, vad händer med värdefunktion?
            Svar: För mig så började det dyka upp många 0.0 värden. Det kan bero på att jag lekte relativt mycket med vinden
        - Prova med 𝛾=0,9, vad händer med värdefunktion?
            Svar: Flesta värden var 0.0, vissa -10 och vissa -9.x. 

In [5]:
# En körning utan att ha ändra vindstyrkan
MC_first_visit()

starting episode 0
starting episode 100
starting episode 200
starting episode 300
starting episode 400


-1421.50	-1421.38	-1422.47	-1395.80	-1380.73	-1358.29	

-1431.07	-1421.06	-1412.46	-1422.03	-1411.63	-1334.13	

-1447.73	-1440.39	-1428.45	-1422.99	-1389.96	-1261.53	

-1440.23	-1442.32	-1413.67	-1436.38	-1335.83	-1067.55	

-1446.94	-1456.22	-1485.62	-1350.55	0.00	-661.06	

-1422.18	-1448.28	-1407.92	0.00	-942.63	-841.90	

--------------------


In [6]:
# Ändrad vindstyrka med en enhet
def MC_first_visitWindy(loop=500):

    grid = WindyGrid(6,6, wind=[0, 0, 2, 3, 2, 0])
    GAMMA = 1.0

    policy = {}                              #  a dictionary that maps each
    for state in grid.stateSpace:            #  state to the list of possible actions
        policy[state] = grid.possibleActions

    V = {}                                   # Initialize our initial estimate of the value
    for state in grid.stateSpacePlus:        # function. Each state gets a value of 0.
        V[state] = 0

    returns = {}                             #Initialize a dictionary that keeps a list
    for state in grid.stateSpace:            #of the returns for each state.
        returns[state] = []

    for i in range(loop):                     # Loop over 500 games,
        observation, done = grid.reset()     # resetting the grid and memory with each game.
        memory = []                          # empty list to keep track of the states visited
        statesReturns = []                   # and returns at each time step
        if i % 100 == 0:                     # Just to know if the game is running.
            print('starting episode', i)
        while not done:                      # While the game isn't done
            # attempt to follow the policy. In this case choose an action
            # according to the random equiprobable strategy.
            action = np.random.choice(policy[observation])
            observation_, reward, done, info = grid.step(action)  # Take that action, get new state, reward and done
            memory.append((observation, action, reward))
            observation = observation_

        # append terminal state
        memory.append((observation, action, reward))

        G = 0                                  # set G=0
        last = True                            # initialize a Boolean to keep track of the visit to the last state
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:                                    # Skip the terminal state and append the set of states
                statesReturns.append((state,G))      #  and returns to the statesReturns list.
            G = GAMMA*G + reward

        statesReturns.reverse()                  # to ge it in chronological order
        statesVisited = []                       # keep track of the visited states during the episode.
        for state, G in statesReturns:
            if state not in statesVisited:       # Iterate over the episode and see
                returns[state].append(G)         # if each state has been visited before.
                V[state] = np.mean(returns[state])
                statesVisited.append(state)

                #If it hasn't, meaning this is the agent's first visit, go ahead and append
                #the returns to the returns dictionary for that state.
                #Calculate the value function by taking the mean of the returns for that state, and finally,
                #append that state to the list of statesVisited.
    print("\n")
    printV(V, grid)

In [7]:
MC_first_visitWindy()

starting episode 0
starting episode 100
starting episode 200
starting episode 300
starting episode 400


-1647.03	-1648.01	-1634.85	-1623.93	-1605.77	-1583.10	

-1640.19	-1633.71	-1631.79	-1641.02	-1584.98	-1557.25	

-1638.22	-1628.86	-1626.52	-1548.67	-1602.21	-1490.50	

-1655.13	-1651.63	-1638.54	-1610.65	-1573.94	-1298.62	

-1651.37	-1632.08	-1606.01	0.00	0.00	-729.89	

-1602.26	-1607.19	-1635.28	0.00	-1061.55	-840.73	

--------------------


In [8]:
# Ändrad Gamma
def MC_first_visitY05(loop=500):

    grid = WindyGrid(6,6, wind=[0, 0, 2, 3, 2, 0])
    GAMMA = 0.5

    policy = {}                              #  a dictionary that maps each
    for state in grid.stateSpace:            #  state to the list of possible actions
        policy[state] = grid.possibleActions

    V = {}                                   # Initialize our initial estimate of the value
    for state in grid.stateSpacePlus:        # function. Each state gets a value of 0.
        V[state] = 0

    returns = {}                             #Initialize a dictionary that keeps a list
    for state in grid.stateSpace:            #of the returns for each state.
        returns[state] = []

    for i in range(loop):                     # Loop over 500 games,
        observation, done = grid.reset()     # resetting the grid and memory with each game.
        memory = []                          # empty list to keep track of the states visited
        statesReturns = []                   # and returns at each time step
        if i % 100 == 0:                     # Just to know if the game is running.
            print('starting episode', i)
        while not done:                      # While the game isn't done
            # attempt to follow the policy. In this case choose an action
            # according to the random equiprobable strategy.
            action = np.random.choice(policy[observation])
            observation_, reward, done, info = grid.step(action)  # Take that action, get new state, reward and done
            memory.append((observation, action, reward))
            observation = observation_

        # append terminal state
        memory.append((observation, action, reward))

        G = 0                                  # set G=0
        last = True                            # initialize a Boolean to keep track of the visit to the last state
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:                                    # Skip the terminal state and append the set of states
                statesReturns.append((state,G))      #  and returns to the statesReturns list.
            G = GAMMA*G + reward

        statesReturns.reverse()                  # to ge it in chronological order
        statesVisited = []                       # keep track of the visited states during the episode.
        for state, G in statesReturns:
            if state not in statesVisited:       # Iterate over the episode and see
                returns[state].append(G)         # if each state has been visited before.
                V[state] = np.mean(returns[state])
                statesVisited.append(state)

                #If it hasn't, meaning this is the agent's first visit, go ahead and append
                #the returns to the returns dictionary for that state.
                #Calculate the value function by taking the mean of the returns for that state, and finally,
                #append that state to the list of statesVisited.
    print("\n")
    printV(V, grid)

In [9]:
MC_first_visitY05()

starting episode 0
starting episode 100
starting episode 200
starting episode 300
starting episode 400


-2.00	-2.00	-2.00	-2.00	-2.00	-2.00	

-2.00	-2.00	-2.00	-2.00	-2.00	-2.00	

-2.00	-2.00	-2.00	-2.00	-2.00	-1.99	

-2.00	-2.00	-2.00	-2.00	-2.00	-1.97	

-2.00	-2.00	-2.00	0.00	0.00	-1.70	

-2.00	-2.00	-2.00	0.00	-1.78	-1.91	

--------------------


In [11]:
# Ändrad Gamma
def MC_first_visitY09(loop=500):

    grid = WindyGrid(6,6, wind=[0, 0, 2, 3, 2, 0])
    GAMMA = 0.9

    policy = {}                              #  a dictionary that maps each
    for state in grid.stateSpace:            #  state to the list of possible actions
        policy[state] = grid.possibleActions

    V = {}                                   # Initialize our initial estimate of the value
    for state in grid.stateSpacePlus:        # function. Each state gets a value of 0.
        V[state] = 0

    returns = {}                             #Initialize a dictionary that keeps a list
    for state in grid.stateSpace:            #of the returns for each state.
        returns[state] = []

    for i in range(loop):                     # Loop over 500 games,
        observation, done = grid.reset()     # resetting the grid and memory with each game.
        memory = []                          # empty list to keep track of the states visited
        statesReturns = []                   # and returns at each time step
        if i % 100 == 0:                     # Just to know if the game is running.
            print('starting episode', i)
        while not done:                      # While the game isn't done
            # attempt to follow the policy. In this case choose an action
            # according to the random equiprobable strategy.
            action = np.random.choice(policy[observation])
            observation_, reward, done, info = grid.step(action)  # Take that action, get new state, reward and done
            memory.append((observation, action, reward))
            observation = observation_

        # append terminal state
        memory.append((observation, action, reward))

        G = 0                                  # set G=0
        last = True                            # initialize a Boolean to keep track of the visit to the last state
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:                                    # Skip the terminal state and append the set of states
                statesReturns.append((state,G))      #  and returns to the statesReturns list.
            G = GAMMA*G + reward

        statesReturns.reverse()                  # to ge it in chronological order
        statesVisited = []                       # keep track of the visited states during the episode.
        for state, G in statesReturns:
            if state not in statesVisited:       # Iterate over the episode and see
                returns[state].append(G)         # if each state has been visited before.
                V[state] = np.mean(returns[state])
                statesVisited.append(state)

                #If it hasn't, meaning this is the agent's first visit, go ahead and append
                #the returns to the returns dictionary for that state.
                #Calculate the value function by taking the mean of the returns for that state, and finally,
                #append that state to the list of statesVisited.
    print("\n")
    printV(V, grid)

In [12]:
MC_first_visitY09()

starting episode 0
starting episode 100
starting episode 200
starting episode 300
starting episode 400


-10.00	-9.99	-9.99	-9.98	-9.96	-9.91	

-10.00	-10.00	-9.99	-9.97	-9.97	-9.83	

-10.00	-10.00	-10.00	-9.96	-9.98	-9.53	

-10.00	-10.00	-10.00	-9.98	-9.94	-8.82	

-10.00	-10.00	-9.98	0.00	0.00	-6.01	

-10.00	-9.99	-9.99	0.00	-6.88	-7.16	

--------------------


In [13]:
# Ändrad Gamma
def MC_first_visitY095(loop=500):

    grid = WindyGrid(6,6, wind=[0, 0, 2, 3, 2, 0])
    GAMMA = 0.9

    policy = {}                              #  a dictionary that maps each
    for state in grid.stateSpace:            #  state to the list of possible actions
        policy[state] = grid.possibleActions

    V = {}                                   # Initialize our initial estimate of the value
    for state in grid.stateSpacePlus:        # function. Each state gets a value of 0.
        V[state] = 0

    returns = {}                             #Initialize a dictionary that keeps a list
    for state in grid.stateSpace:            #of the returns for each state.
        returns[state] = []

    for i in range(loop):                     # Loop over 500 games,
        observation, done = grid.reset()     # resetting the grid and memory with each game.
        memory = []                          # empty list to keep track of the states visited
        statesReturns = []                   # and returns at each time step
        if i % 100 == 0:                     # Just to know if the game is running.
            print('starting episode', i)
        while not done:                      # While the game isn't done
            # attempt to follow the policy. In this case choose an action
            # according to the random equiprobable strategy.
            action = np.random.choice(policy[observation])
            observation_, reward, done, info = grid.step(action)  # Take that action, get new state, reward and done
            memory.append((observation, action, reward))
            observation = observation_

        # append terminal state
        memory.append((observation, action, reward))

        G = 0                                  # set G=0
        last = True                            # initialize a Boolean to keep track of the visit to the last state
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:                                    # Skip the terminal state and append the set of states
                statesReturns.append((state,G))      #  and returns to the statesReturns list.
            G = GAMMA*G + reward

        statesReturns.reverse()                  # to ge it in chronological order
        statesVisited = []                       # keep track of the visited states during the episode.
        for state, G in statesReturns:
            if state not in statesVisited:       # Iterate over the episode and see
                returns[state].append(G)         # if each state has been visited before.
                V[state] = np.mean(returns[state])
                statesVisited.append(state)

                #If it hasn't, meaning this is the agent's first visit, go ahead and append
                #the returns to the returns dictionary for that state.
                #Calculate the value function by taking the mean of the returns for that state, and finally,
                #append that state to the list of statesVisited.
    print("\n")
    printV(V, grid)

In [14]:
MC_first_visitY095()

starting episode 0
starting episode 100
starting episode 200
starting episode 300
starting episode 400


-9.99	-9.98	-9.98	-9.94	-9.91	-9.85	

-9.99	-9.98	-9.97	-9.97	-9.95	-9.69	

-9.99	-9.98	-9.98	-9.96	-9.96	-9.36	

-9.99	-9.99	-9.99	-9.97	-9.90	-8.49	

-9.99	-9.99	-9.97	0.00	0.00	-5.41	

-9.99	-9.98	-9.98	0.00	-6.93	-7.09	

--------------------


In [23]:
def MC_first_visit8x8(loop=500):

    grid = WindyGrid(8,8, wind=[0, 0, 2, 3, 2, 1, 0, 0])
    GAMMA = 0.9

    policy = {}                              #  a dictionary that maps each
    for state in grid.stateSpace:            #  state to the list of possible actions
        policy[state] = grid.possibleActions

    V = {}                                   # Initialize our initial estimate of the value
    for state in grid.stateSpacePlus:        # function. Each state gets a value of 0.
        V[state] = 0

    returns = {}                             #Initialize a dictionary that keeps a list
    for state in grid.stateSpace:            #of the returns for each state.
        returns[state] = []

    for i in range(loop):                     # Loop over 500 games,
        observation, done = grid.reset()     # resetting the grid and memory with each game.
        memory = []                          # empty list to keep track of the states visited
        statesReturns = []                   # and returns at each time step
        if i % 100 == 0:                     # Just to know if the game is running.
            print('starting episode', i)
        while not done:                      # While the game isn't done
            # attempt to follow the policy. In this case choose an action
            # according to the random equiprobable strategy.
            action = np.random.choice(policy[observation])
            observation_, reward, done, info = grid.step(action)  # Take that action, get new state, reward and done
            memory.append((observation, action, reward))
            observation = observation_

        # append terminal state
        memory.append((observation, action, reward))

        G = 0                                  # set G=0
        last = True                            # initialize a Boolean to keep track of the visit to the last state
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:                                    # Skip the terminal state and append the set of states
                statesReturns.append((state,G))      #  and returns to the statesReturns list.
            G = GAMMA*G + reward

        statesReturns.reverse()                  # to ge it in chronological order
        statesVisited = []                       # keep track of the visited states during the episode.
        for state, G in statesReturns:
            if state not in statesVisited:       # Iterate over the episode and see
                returns[state].append(G)         # if each state has been visited before.
                V[state] = np.mean(returns[state])
                statesVisited.append(state)

                #If it hasn't, meaning this is the agent's first visit, go ahead and append
                #the returns to the returns dictionary for that state.
                #Calculate the value function by taking the mean of the returns for that state, and finally,
                #append that state to the list of statesVisited.
    print("\n")
    printV(V, grid)

In [24]:
MC_first_visit8x8()

starting episode 0
starting episode 100
starting episode 200
starting episode 300
starting episode 400


-10.00	-10.00	-10.00	-10.00	-10.00	-9.99	-9.98	-9.97	

-10.00	-10.00	-10.00	-10.00	-9.99	-9.98	-9.98	-9.95	

-10.00	-10.00	-10.00	-10.00	-9.99	-9.99	-9.94	-9.90	

-10.00	-10.00	-10.00	-10.00	0.00	-9.93	-9.67	-9.80	

-10.00	-10.00	-10.00	-10.00	-7.73	-7.07	-9.12	-9.54	

-10.00	-10.00	-10.00	-10.00	-9.46	-9.27	-9.33	-9.45	

-10.00	-10.00	-10.00	0.00	-6.82	-8.89	-9.33	-9.54	

-10.00	-10.00	-10.00	0.00	0.00	-8.77	-9.29	-9.47	

--------------------


In [6]:
def MC_first_visit10x10(loop=500):

    grid = WindyGrid(10,10, wind=[1, 1, 1, 2, 3, 3, 1, 1, 1, 0])
    GAMMA = 0.9

    policy = {}                              #  a dictionary that maps each
    for state in grid.stateSpace:            #  state to the list of possible actions
        policy[state] = grid.possibleActions

    V = {}                                   # Initialize our initial estimate of the value
    for state in grid.stateSpacePlus:        # function. Each state gets a value of 0.
        V[state] = 0

    returns = {}                             #Initialize a dictionary that keeps a list
    for state in grid.stateSpace:            #of the returns for each state.
        returns[state] = []

    for i in range(loop):                     # Loop over 500 games,
        observation, done = grid.reset()     # resetting the grid and memory with each game.
        memory = []                          # empty list to keep track of the states visited
        statesReturns = []                   # and returns at each time step
        if i % 100 == 0:                     # Just to know if the game is running.
            print('starting episode', i)
        while not done:                      # While the game isn't done
            # attempt to follow the policy. In this case choose an action
            # according to the random equiprobable strategy.
            action = np.random.choice(policy[observation])
            observation_, reward, done, info = grid.step(action)  # Take that action, get new state, reward and done
            memory.append((observation, action, reward))
            observation = observation_

        # append terminal state
        memory.append((observation, action, reward))

        G = 0                                  # set G=0
        last = True                            # initialize a Boolean to keep track of the visit to the last state
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:                                    # Skip the terminal state and append the set of states
                statesReturns.append((state,G))      #  and returns to the statesReturns list.
            G = GAMMA*G + reward

        statesReturns.reverse()                  # to ge it in chronological order
        statesVisited = []                       # keep track of the visited states during the episode.
        for state, G in statesReturns:
            if state not in statesVisited:       # Iterate over the episode and see
                returns[state].append(G)         # if each state has been visited before.
                V[state] = np.mean(returns[state])
                statesVisited.append(state)

                #If it hasn't, meaning this is the agent's first visit, go ahead and append
                #the returns to the returns dictionary for that state.
                #Calculate the value function by taking the mean of the returns for that state, and finally,
                #append that state to the list of statesVisited.
    print("\n")
    printV(V, grid)

In [7]:
MC_first_visit10x10()

starting episode 0
starting episode 100
starting episode 200
starting episode 300
starting episode 400


-10.00	-10.00	-9.99	-9.98	-9.98	-9.95	-9.92	-9.84	-9.64	-9.32	

0.00	0.00	0.00	0.00	0.00	-9.99	-9.94	-9.68	-9.66	-8.72	

0.00	0.00	0.00	0.00	0.00	-10.00	-9.28	-9.77	0.00	-6.14	

0.00	0.00	0.00	0.00	0.00	-10.00	-10.00	-7.08	-8.83	-7.74	

0.00	0.00	0.00	0.00	0.00	-10.00	-7.30	-9.76	-5.71	-7.67	

0.00	0.00	0.00	0.00	0.00	0.00	-10.00	-8.14	-8.46	-8.28	

0.00	0.00	0.00	0.00	0.00	0.00	0.00	-10.00	-8.01	-8.39	

0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	-7.39	-8.67	

0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	-10.00	-9.31	

0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	-8.95	-9.24	

--------------------


## Exploring Start Monte Carlo

In [9]:
def MC_exploring_starts():
    grid = WindyGrid(6,6, wind=[0, 0, 1, 2, 1, 0])
    GAMMA = 0.9

    # Initialize Q, returns, and pairs visited
    Q = {}
    returns = {}
    pairsVisited = {}
    for state in grid.stateSpacePlus:
        for action in grid.possibleActions:
            Q[(state, action)] = 0
            returns[(state,action)] = 0
            pairsVisited[(state,action)] = 0

    # initialize a random policy
    policy = {}
    for state in grid.stateSpace:
        policy[state] = np.random.choice(grid.possibleActions)

    for i in range(1000000):
        if i % 50000 == 0:
            print('starting episode', i)
        statesActionsReturns = []
        observation = np.random.choice(grid.stateSpace)
        action = np.random.choice(grid.possibleActions)
        grid.setState(observation)
        observation_, reward, done, info = grid.step(action)
        memory = [(observation, action, reward)]
        steps = 1
        while not done:
            action = policy[observation_]
            steps += 1
            observation, reward, done, info = grid.step(action)
            if steps > 15 and not done:
                done = True
                reward = -steps
            memory.append((observation_, action, reward))
            observation_ = observation

        # append the terminal state
        memory.append((observation_, action, reward))

        G = 0
        last = True # start at t = T - 1
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:
                statesActionsReturns.append((state,action, G))
            G = GAMMA*G + reward

        statesActionsReturns.reverse()
        statesAndActions = []
        for state, action, G in statesActionsReturns:
            if (state, action) not in statesAndActions:
                pairsVisited[(state,action)] += 1
                returns[(state,action)] += (1 / pairsVisited[(state,action)])*(G-returns[(state,action)])
                Q[(state,action)] = returns[(state,action)]
                statesAndActions.append((state,action))
                values = np.array([Q[(state,a)] for a in grid.possibleActions])
                best = np.argmax(values)
                policy[state] = grid.possibleActions[best]

    printQ(Q, grid)
    printPolicy(policy,grid)

## Del 2


- Använd  *exploring starts* Monte Carlo Metoden

1. Öka vindstyrkan med en enhet.
    - Hur ändras slutvärdesfunktionen?
        Svar: på ett mindre drastiskt sätt en de tidigare MC funktionerna. Detta kan bero på att jag inte ökat vinden tillräckligt men körningstiden blir så drastistk mycket längre alltid vid en stor ökning


2. Hur ändras policyn om man ändra gamma till:
    - 𝛾=0.5 Svar: Värdena ändrades till -1.99 eller något annat värde där nära
    - 𝛾=0,9 Svar: Det ursprungliga körningen var i -0.9 så inget att tillägga
    - 𝛾=0,95 Svar: Värdena förskjöt sig till -8 värden.


3. Testa rutnätsvärlden i storlekarna:
    - 8x8
        - Ändra på vinden, vad händer med policyn?
             Svar: Policyn håller förvånansvärt bra fastän det är ett större fält och ökad vind
        - Prova med 𝛾=0,9, vad händer med policyn?
             Svar: gamman var 0.9 då på förra körningen eftersom denna funktion presenterades ursprungligen med gamma 0.9
             GAMMA: 1
             Svar: Med gamma 1 klarar sig policyn sämre än med 0.9
    - 10x10
        - Ändra på vinden, vad händer med policyn?
            Svar: Policyn klarar sig ännu men jag gjorde vinden mycket värre vilket resulterade i sämre resultat

In [10]:
MC_exploring_starts()

starting episode 0
starting episode 50000
starting episode 100000
starting episode 150000
starting episode 200000
starting episode 250000
starting episode 300000
starting episode 350000
starting episode 400000
starting episode 450000
starting episode 500000
starting episode 550000
starting episode 600000
starting episode 650000
starting episode 700000
starting episode 750000
starting episode 800000
starting episode 850000
starting episode 900000
starting episode 950000
[-6.88523, -6.91967, -6.88524, -6.51754]	[-6.53039, -6.55184, -6.90646, -6.12736]	[-6.13409, -6.15225, -6.5386, -5.69549]	[-5.69694, -5.70237, -6.13712, -5.21726]	[-5.22435, -5.22, -5.70178, -4.68576]	[-4.69032, -4.09514, -5.21859, -4.69472]	

[-6.88971, -6.86558, -6.88042, -6.53605]	[-6.54246, -6.51596, -6.87907, -6.12734]	[-6.13231, -6.13546, -6.53278, -5.69545]	[-5.70735, -5.69795, -6.13115, -5.21711]	[-5.22104, -5.22177, -5.6981, -4.68576]	[-4.68998, -3.43903, -5.21919, -4.10001]	

[-6.88136, -6.86786, -6.86913, -6.5

In [13]:
def MC_exploring_startsWindy():
    grid = WindyGrid(6,6, wind=[1, 0, 2, 2, 1, 0])
    GAMMA = 0.9

    # Initialize Q, returns, and pairs visited
    Q = {}
    returns = {}
    pairsVisited = {}
    for state in grid.stateSpacePlus:
        for action in grid.possibleActions:
            Q[(state, action)] = 0
            returns[(state,action)] = 0
            pairsVisited[(state,action)] = 0

    # initialize a random policy
    policy = {}
    for state in grid.stateSpace:
        policy[state] = np.random.choice(grid.possibleActions)

    for i in range(1000000):
        if i % 50000 == 0:
            print('starting episode', i)
        statesActionsReturns = []
        observation = np.random.choice(grid.stateSpace)
        action = np.random.choice(grid.possibleActions)
        grid.setState(observation)
        observation_, reward, done, info = grid.step(action)
        memory = [(observation, action, reward)]
        steps = 1
        while not done:
            action = policy[observation_]
            steps += 1
            observation, reward, done, info = grid.step(action)
            if steps > 15 and not done:
                done = True
                reward = -steps
            memory.append((observation_, action, reward))
            observation_ = observation

        # append the terminal state
        memory.append((observation_, action, reward))

        G = 0
        last = True # start at t = T - 1
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:
                statesActionsReturns.append((state,action, G))
            G = GAMMA*G + reward

        statesActionsReturns.reverse()
        statesAndActions = []
        for state, action, G in statesActionsReturns:
            if (state, action) not in statesAndActions:
                pairsVisited[(state,action)] += 1
                returns[(state,action)] += (1 / pairsVisited[(state,action)])*(G-returns[(state,action)])
                Q[(state,action)] = returns[(state,action)]
                statesAndActions.append((state,action))
                values = np.array([Q[(state,a)] for a in grid.possibleActions])
                best = np.argmax(values)
                policy[state] = grid.possibleActions[best]

    printQ(Q, grid)
    printPolicy(policy,grid)

In [14]:
MC_exploring_startsWindy()

starting episode 0
starting episode 50000
starting episode 100000
starting episode 150000
starting episode 200000
starting episode 250000
starting episode 300000
starting episode 350000
starting episode 400000
starting episode 450000
starting episode 500000
starting episode 550000
starting episode 600000
starting episode 650000
starting episode 700000
starting episode 750000
starting episode 800000
starting episode 850000
starting episode 900000
starting episode 950000
[-6.98022, -6.97757, -6.95203, -6.61318]	[-6.62032, -6.51804, -6.9491, -6.13324]	[-6.28536, -6.16384, -6.62673, -5.69719]	[-5.71805, -5.77205, -6.1637, -5.21919]	[-5.23357, -5.26041, -5.80792, -4.68572]	[-4.71968, -4.09527, -5.22577, -4.71149]	

[-7.00309, -6.96762, -6.97697, -6.59739]	[-6.61685, -6.53653, -6.97763, -6.12816]	[-6.14266, -6.15194, -6.63328, -5.69616]	[-5.74923, -5.74459, -6.13071, -5.21807]	[-5.22731, -5.22551, -5.70289, -4.68977]	[-4.69456, -3.43906, -5.23433, -4.1155]	

[-6.96811, -6.89149, -6.88306, -6

In [17]:
def MC_exploring_startsY():
    grid = WindyGrid(6,6, wind=[1, 0, 2, 2, 1, 0])
    GAMMA = 0.95

    # Initialize Q, returns, and pairs visited
    Q = {}
    returns = {}
    pairsVisited = {}
    for state in grid.stateSpacePlus:
        for action in grid.possibleActions:
            Q[(state, action)] = 0
            returns[(state,action)] = 0
            pairsVisited[(state,action)] = 0

    # initialize a random policy
    policy = {}
    for state in grid.stateSpace:
        policy[state] = np.random.choice(grid.possibleActions)

    for i in range(1000000):
        if i % 50000 == 0:
            print('starting episode', i)
        statesActionsReturns = []
        observation = np.random.choice(grid.stateSpace)
        action = np.random.choice(grid.possibleActions)
        grid.setState(observation)
        observation_, reward, done, info = grid.step(action)
        memory = [(observation, action, reward)]
        steps = 1
        while not done:
            action = policy[observation_]
            steps += 1
            observation, reward, done, info = grid.step(action)
            if steps > 15 and not done:
                done = True
                reward = -steps
            memory.append((observation_, action, reward))
            observation_ = observation

        # append the terminal state
        memory.append((observation_, action, reward))

        G = 0
        last = True # start at t = T - 1
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:
                statesActionsReturns.append((state,action, G))
            G = GAMMA*G + reward

        statesActionsReturns.reverse()
        statesAndActions = []
        for state, action, G in statesActionsReturns:
            if (state, action) not in statesAndActions:
                pairsVisited[(state,action)] += 1
                returns[(state,action)] += (1 / pairsVisited[(state,action)])*(G-returns[(state,action)])
                Q[(state,action)] = returns[(state,action)]
                statesAndActions.append((state,action))
                values = np.array([Q[(state,a)] for a in grid.possibleActions])
                best = np.argmax(values)
                policy[state] = grid.possibleActions[best]

    printQ(Q, grid)
    printPolicy(policy,grid)

In [18]:
MC_exploring_startsY()

starting episode 0
starting episode 50000
starting episode 100000
starting episode 150000
starting episode 200000
starting episode 250000
starting episode 300000
starting episode 350000
starting episode 400000
starting episode 450000
starting episode 500000
starting episode 550000
starting episode 600000
starting episode 650000
starting episode 700000
starting episode 750000
starting episode 800000
starting episode 850000
starting episode 900000
starting episode 950000
[-8.68882, -8.77179, -8.70482, -8.07249]	[-8.08603, -8.03202, -8.74434, -7.39811]	[-7.47249, -7.40802, -8.09801, -6.73243]	[-6.77196, -6.75022, -7.40541, -6.03424]	[-6.03615, -6.05034, -6.74237, -5.29885]	[-5.34361, -4.52474, -6.04158, -5.30106]	

[-8.69202, -8.68473, -8.67995, -8.07283]	[-8.0979, -8.04789, -8.67413, -7.39526]	[-7.40024, -7.41025, -8.09323, -6.73229]	[-6.73427, -6.75824, -7.40516, -6.03377]	[-6.03883, -6.03873, -6.74724, -5.29932]	[-5.30405, -3.71003, -6.04175, -4.55424]	

[-8.68889, -8.63114, -8.62871, 

In [23]:
def MC_exploring_starts8x8():
    grid = WindyGrid(8,8, wind=[1, 1, 0, 2, 2, 1, 0, 0])
    GAMMA = 1

    # Initialize Q, returns, and pairs visited
    Q = {}
    returns = {}
    pairsVisited = {}
    for state in grid.stateSpacePlus:
        for action in grid.possibleActions:
            Q[(state, action)] = 0
            returns[(state,action)] = 0
            pairsVisited[(state,action)] = 0

    # initialize a random policy
    policy = {}
    for state in grid.stateSpace:
        policy[state] = np.random.choice(grid.possibleActions)

    for i in range(1000000):
        if i % 50000 == 0:
            print('starting episode', i)
        statesActionsReturns = []
        observation = np.random.choice(grid.stateSpace)
        action = np.random.choice(grid.possibleActions)
        grid.setState(observation)
        observation_, reward, done, info = grid.step(action)
        memory = [(observation, action, reward)]
        steps = 1
        while not done:
            action = policy[observation_]
            steps += 1
            observation, reward, done, info = grid.step(action)
            if steps > 15 and not done:
                done = True
                reward = -steps
            memory.append((observation_, action, reward))
            observation_ = observation

        # append the terminal state
        memory.append((observation_, action, reward))

        G = 0
        last = True # start at t = T - 1
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:
                statesActionsReturns.append((state,action, G))
            G = GAMMA*G + reward

        statesActionsReturns.reverse()
        statesAndActions = []
        for state, action, G in statesActionsReturns:
            if (state, action) not in statesAndActions:
                pairsVisited[(state,action)] += 1
                returns[(state,action)] += (1 / pairsVisited[(state,action)])*(G-returns[(state,action)])
                Q[(state,action)] = returns[(state,action)]
                statesAndActions.append((state,action))
                values = np.array([Q[(state,a)] for a in grid.possibleActions])
                best = np.argmax(values)
                policy[state] = grid.possibleActions[best]

    printQ(Q, grid)
    printPolicy(policy,grid)

In [24]:
MC_exploring_starts8x8()

starting episode 0
starting episode 50000
starting episode 100000
starting episode 150000
starting episode 200000
starting episode 250000
starting episode 300000
starting episode 350000
starting episode 400000
starting episode 450000
starting episode 500000
starting episode 550000
starting episode 600000
starting episode 650000
starting episode 700000
starting episode 750000
starting episode 800000
starting episode 850000
starting episode 900000
starting episode 950000
[-10.09126, -10.09173, -10.11739, -9.0791]	[-10.86893, -9.06456, -10.09606, -8.00301]	[-8.04719, -7.00019, -9.74783, -9.13251]	[-9.55446, -9.1577, -8.00704, -9.55004]	[-9.09491, -9.36416, -9.08037, -8.11681]	[-8.77592, -8.11161, -9.063, -7.06666]	[-7.07723, -6.0017, -8.20066, -8.0093]	[-8.01918, -7.00318, -7.09322, -8.00942]	

[-10.07639, -10.31187, -10.35106, -9.06863]	[-9.04727, -9.05488, -10.22791, -8.00519]	[-8.06286, -6.00014, -9.06438, -9.61685]	[-9.46856, -9.14131, -8.14463, -9.10225]	[-9.37232, -9.10549, -9.19112

In [28]:
def MC_exploring_starts10x10():
    grid = WindyGrid(10,10, wind=[2, 1, 1, 0, 3, 2, 2, 0, 0, 0])
    GAMMA = 1

    # Initialize Q, returns, and pairs visited
    Q = {}
    returns = {}
    pairsVisited = {}
    for state in grid.stateSpacePlus:
        for action in grid.possibleActions:
            Q[(state, action)] = 0
            returns[(state,action)] = 0
            pairsVisited[(state,action)] = 0

    # initialize a random policy
    policy = {}
    for state in grid.stateSpace:
        policy[state] = np.random.choice(grid.possibleActions)

    for i in range(1000000):
        if i % 50000 == 0:
            print('starting episode', i)
        statesActionsReturns = []
        observation = np.random.choice(grid.stateSpace)
        action = np.random.choice(grid.possibleActions)
        grid.setState(observation)
        observation_, reward, done, info = grid.step(action)
        memory = [(observation, action, reward)]
        steps = 1
        while not done:
            action = policy[observation_]
            steps += 1
            observation, reward, done, info = grid.step(action)
            if steps > 15 and not done:
                done = True
                reward = -steps
            memory.append((observation_, action, reward))
            observation_ = observation

        # append the terminal state
        memory.append((observation_, action, reward))

        G = 0
        last = True # start at t = T - 1
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:
                statesActionsReturns.append((state,action, G))
            G = GAMMA*G + reward

        statesActionsReturns.reverse()
        statesAndActions = []
        for state, action, G in statesActionsReturns:
            if (state, action) not in statesAndActions:
                pairsVisited[(state,action)] += 1
                returns[(state,action)] += (1 / pairsVisited[(state,action)])*(G-returns[(state,action)])
                Q[(state,action)] = returns[(state,action)]
                statesAndActions.append((state,action))
                values = np.array([Q[(state,a)] for a in grid.possibleActions])
                best = np.argmax(values)
                policy[state] = grid.possibleActions[best]

    printQ(Q, grid)
    printPolicy(policy,grid)

In [29]:
MC_exploring_starts10x10()

starting episode 0
starting episode 50000
starting episode 100000
starting episode 150000
starting episode 200000
starting episode 250000
starting episode 300000
starting episode 350000
starting episode 400000
starting episode 450000
starting episode 500000
starting episode 550000
starting episode 600000
starting episode 650000
starting episode 700000
starting episode 750000
starting episode 800000
starting episode 850000
starting episode 900000
starting episode 950000
[-11.09406, -11.98148, -11.06884, -10.00631]	[-10.07271, -11.33742, -11.05663, -9.00867]	[-9.0294, -9.62717, -10.33788, -8.00081]	[-8.04399, -9.05866, -9.35886, -7.00042]	[-7.36094, -7.03047, -8.02942, -6.00069]	[-6.03185, -6.53927, -7.04151, -5.00073]	[-5.01637, -5.3627, -6.06386, -4.00008]	[-4.01515, -3.01659, -5.11724, -3.00013]	[-3.01559, -2.00006, -4.01623, -4.01571]	[-4.0157, -3.01717, -3.00313, -4.01615]	

[-11.11339, -11.0705, -11.10983, -10.04411]	[-10.02723, -10.284, -11.09818, -9.00338]	[-9.05691, -9.07327, -1

## On-policy first visit Monte Carlo for $\varepsilon$-soft policies

In [31]:
def MC_without_exploring_starts():
    grid = WindyGrid(6,6, wind=[0, 0, 1, 2, 1, 0])
    GAMMA = 0.9
    EPS = 0.4

    Q = {}
    returns = {}
    pairsVisited = {}
    for state in grid.stateSpacePlus:
        for action in grid.actionSpace.keys():
            Q[(state, action)] = 0
            returns[(state,action)] = 0
            pairsVisited[(state,action)] = 0

    policy = {}
    for state in grid.stateSpace:
        policy[state] = grid.possibleActions

    for i in range(1000000):
        statesActionsReturns = []
        if i % 100000 == 0:
            print('starting episode', i)
        observation, done = grid.reset()
        memory = []
        steps = 0
        while not done:
            if len(policy[observation]) > 1:
                action = np.random.choice(policy[observation])
            else:
                action = policy[observation]
            observation_, reward, done, info = grid.step(action)
            steps += 1
            if steps > 25 and not done:
                done = True
                reward = -steps
            memory.append((observation, action, reward))
            observation = observation_

        #append the terminal state
        memory.append((observation, action, reward))

        G = 0
        last = True # start at t = T - 1
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:
                statesActionsReturns.append((state,action,G))
            G = GAMMA*G + reward
        statesActionsReturns.reverse()

        statesAndActions = []
        for state, action, G in statesActionsReturns:
            if (state, action) not in statesAndActions:
                pairsVisited[(state,action)] += 1
                returns[(state,action)] += (1 / pairsVisited[(state,action)])*(G-returns[(state,action)])
                Q[(state,action)] = returns[(state,action)]
                statesAndActions.append((state,action))
                values = np.array([Q[(state,a)] for a in grid.possibleActions])
                best = np.random.choice(np.where(values==values.max())[0])
                rand = np.random.random()
                if rand < 1 - EPS:
                    policy[state] = grid.possibleActions[best]
                else:
                    policy[state] = np.random.choice(grid.possibleActions)

    printQ(Q, grid)
    printPolicy(policy,grid)

In [32]:
MC_without_exploring_starts()

starting episode 0
starting episode 100000
starting episode 200000
starting episode 300000
starting episode 400000
starting episode 500000
starting episode 600000
starting episode 700000
starting episode 800000
starting episode 900000
[-13.14286, -13.1429, -13.1428, -13.1428]	[-13.54723, -13.55092, -13.54807, -13.54381]	[-13.96813, -13.98302, -13.97508, -13.96125]	[-14.40207, -14.42335, -14.42212, -14.43105]	[-14.91357, -15.07811, -14.9474, -15.05488]	[-15.73333, -16.53104, -15.63402, -15.49425]	

[-13.54366, -13.53334, -13.51398, -13.53008]	[-13.90587, -13.92111, -13.90642, -13.9107]	[-14.39515, -14.37905, -14.48687, -14.38349]	[-15.99043, -16.07759, -16.02481, -15.59127]	[-16.81417, -18.60355, -16.57081, -17.38503]	[-15.91373, -27.07582, -16.06786, -16.1119]	

[-13.99758, -13.966, -13.97966, -13.97004]	[-14.39928, -14.40361, -14.44512, -14.46418]	[-14.98014, -15.00203, -15.45115, -15.0029]	[-16.02382, -16.18954, -17.41623, -16.33156]	[0, 0, -18.11211, 0]	[0, -26.0, 0, -16.57081]	

[-

## Del 3
- Använd *without exploring starts* Monte Carlo Metoden

1. Öka vindstyrkan med en enhet.
    - Hur ändras slutvärdesfunktionen?
        Svar: Förväntat klarar sig agenten sämre, men endo relativt bra

2. Hur ändras policyn om man ändra gamma till:
    - 𝛾=0.5 Svar: Så gott som alla svar är omkring -2 i värde med vissa decimalers skillnad
    - 𝛾=0,9 Svar: Relativt dåliga resultat med t.om. -26
    - 𝛾=0,95 Svar: Väldigt dåliga resultat t.om. -36


3. Testa rutnätsvärlden i storlekarna:
    - 8x8
        - Ändra på vinden, vad händer med policyn?
            Svar: Det dök upp väldigt många 0.00 värden
        - Prova med 𝛾=0,9, vad händer med policyn?
            Svar: Koden var igen ursprungligen med gamma 0.9 så jag jämför istället med 1.0
            1.0: Flera 0 värden finns men den förvånande aspekten är att gamma 1.0 fungerar otroligt dåligt jämfört med   andra parameter specifikationer
    - 10x10
        - Ändra på vinden, vad händer med policyn?
            Svar: Med en gamma på 0.9 och relativt stark vind klarar sig policyn väldigt bra.

In [33]:
def MC_without_exploring_startsWind():
    grid = WindyGrid(6,6, wind=[1, 0, 2, 2, 2, 0])
    GAMMA = 0.9
    EPS = 0.4

    Q = {}
    returns = {}
    pairsVisited = {}
    for state in grid.stateSpacePlus:
        for action in grid.actionSpace.keys():
            Q[(state, action)] = 0
            returns[(state,action)] = 0
            pairsVisited[(state,action)] = 0

    policy = {}
    for state in grid.stateSpace:
        policy[state] = grid.possibleActions

    for i in range(1000000):
        statesActionsReturns = []
        if i % 100000 == 0:
            print('starting episode', i)
        observation, done = grid.reset()
        memory = []
        steps = 0
        while not done:
            if len(policy[observation]) > 1:
                action = np.random.choice(policy[observation])
            else:
                action = policy[observation]
            observation_, reward, done, info = grid.step(action)
            steps += 1
            if steps > 25 and not done:
                done = True
                reward = -steps
            memory.append((observation, action, reward))
            observation = observation_

        #append the terminal state
        memory.append((observation, action, reward))

        G = 0
        last = True # start at t = T - 1
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:
                statesActionsReturns.append((state,action,G))
            G = GAMMA*G + reward
        statesActionsReturns.reverse()

        statesAndActions = []
        for state, action, G in statesActionsReturns:
            if (state, action) not in statesAndActions:
                pairsVisited[(state,action)] += 1
                returns[(state,action)] += (1 / pairsVisited[(state,action)])*(G-returns[(state,action)])
                Q[(state,action)] = returns[(state,action)]
                statesAndActions.append((state,action))
                values = np.array([Q[(state,a)] for a in grid.possibleActions])
                best = np.random.choice(np.where(values==values.max())[0])
                rand = np.random.random()
                if rand < 1 - EPS:
                    policy[state] = grid.possibleActions[best]
                else:
                    policy[state] = np.random.choice(grid.possibleActions)

    printQ(Q, grid)
    printPolicy(policy,grid)

In [34]:
MC_without_exploring_startsWind()

starting episode 0
starting episode 100000
starting episode 200000
starting episode 300000
starting episode 400000
starting episode 500000
starting episode 600000
starting episode 700000
starting episode 800000
starting episode 900000
[-13.14281, -13.1428, -13.1428, -13.14283]	[-13.49254, -13.49264, -13.49455, -13.49205]	[-13.89897, -13.90395, -13.90358, -13.90266]	[-14.3826, -14.39528, -14.37911, -14.38983]	[-15.07536, -15.06862, -14.87732, -14.98252]	[-18.84225, -21.17242, -16.3337, -15.4252]	

[-14.33971, -14.47259, -14.47906, -14.54685]	[-13.88, -13.88698, -13.89344, -13.88]	[-14.34065, -14.33678, -14.33514, -14.33557]	[-15.91373, 0, -15.91373, -23.42625]	[-16.57081, -17.3009, -38.7226, -28.8449]	[-15.91373, -28.91386, -15.91373, -26.0]	

[-14.79881, -15.33013, -15.61206, -15.10968]	[-14.31111, -14.38637, -14.31111, -14.31111]	[-14.79012, -14.79012, -14.79012, -14.82757]	[0, -28.8449, 0, 0]	[0, 0, 0, 0]	[-49.4, -16.57081, 0, -45.46]	

[-18.04567, -16.97936, -18.74369, -15.32236]	[-

In [39]:
def MC_without_exploring_startsY():
    grid = WindyGrid(6,6, wind=[1, 0, 2, 2, 2, 0])
    GAMMA = 0.95
    EPS = 0.4

    Q = {}
    returns = {}
    pairsVisited = {}
    for state in grid.stateSpacePlus:
        for action in grid.actionSpace.keys():
            Q[(state, action)] = 0
            returns[(state,action)] = 0
            pairsVisited[(state,action)] = 0

    policy = {}
    for state in grid.stateSpace:
        policy[state] = grid.possibleActions

    for i in range(1000000):
        statesActionsReturns = []
        if i % 100000 == 0:
            print('starting episode', i)
        observation, done = grid.reset()
        memory = []
        steps = 0
        while not done:
            if len(policy[observation]) > 1:
                action = np.random.choice(policy[observation])
            else:
                action = policy[observation]
            observation_, reward, done, info = grid.step(action)
            steps += 1
            if steps > 25 and not done:
                done = True
                reward = -steps
            memory.append((observation, action, reward))
            observation = observation_

        #append the terminal state
        memory.append((observation, action, reward))

        G = 0
        last = True # start at t = T - 1
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:
                statesActionsReturns.append((state,action,G))
            G = GAMMA*G + reward
        statesActionsReturns.reverse()

        statesAndActions = []
        for state, action, G in statesActionsReturns:
            if (state, action) not in statesAndActions:
                pairsVisited[(state,action)] += 1
                returns[(state,action)] += (1 / pairsVisited[(state,action)])*(G-returns[(state,action)])
                Q[(state,action)] = returns[(state,action)]
                statesAndActions.append((state,action))
                values = np.array([Q[(state,a)] for a in grid.possibleActions])
                best = np.random.choice(np.where(values==values.max())[0])
                rand = np.random.random()
                if rand < 1 - EPS:
                    policy[state] = grid.possibleActions[best]
                else:
                    policy[state] = np.random.choice(grid.possibleActions)

    printQ(Q, grid)
    printPolicy(policy,grid)

In [40]:
MC_without_exploring_startsY()

starting episode 0
starting episode 100000
starting episode 200000
starting episode 300000
starting episode 400000
starting episode 500000
starting episode 600000
starting episode 700000
starting episode 800000
starting episode 900000
[-28.96406, -28.96419, -28.96406, -28.0196]	[-29.43589, -28.32441, -29.43587, -28.08225]	[-30.02404, -30.02275, -30.02681, -28.0107]	[-30.63582, -30.63629, -30.63689, -27.52973]	[-31.1963, -31.19483, -31.20165, -26.51767]	[-31.78255, -24.77834, -31.78318, -31.78622]	

[-30.46417, -30.45967, -30.45843, -30.46048]	[-29.93366, -29.00832, -29.93308, -28.37951]	[-30.47579, -29.16025, -30.48091, -28.34663]	[-34.52065, -34.57104, -34.91663, -34.50956]	[-33.3402, -33.29353, -33.27864, -33.3687]	[-32.4034, -21.87538, -32.40396, -32.4036]	

[-31.01869, -31.03836, -31.07185, -31.00859]	[-30.45524, -29.63744, -30.45524, -29.09742]	[-29.556, -29.91686, -31.03014, -29.07811]	[-35.8426, -36.65764, -35.1748, -35.80905]	[-34.25323, -34.26584, -34.3494, -34.35001]	[-33.047

In [43]:
def MC_without_exploring_starts8x8():
    grid = WindyGrid(8,8, wind=[1, 2, 0, 3, 2, 1, 0, 0])
    GAMMA = 1
    EPS = 0.4

    Q = {}
    returns = {}
    pairsVisited = {}
    for state in grid.stateSpacePlus:
        for action in grid.actionSpace.keys():
            Q[(state, action)] = 0
            returns[(state,action)] = 0
            pairsVisited[(state,action)] = 0

    policy = {}
    for state in grid.stateSpace:
        policy[state] = grid.possibleActions

    for i in range(1000000):
        statesActionsReturns = []
        if i % 100000 == 0:
            print('starting episode', i)
        observation, done = grid.reset()
        memory = []
        steps = 0
        while not done:
            if len(policy[observation]) > 1:
                action = np.random.choice(policy[observation])
            else:
                action = policy[observation]
            observation_, reward, done, info = grid.step(action)
            steps += 1
            if steps > 25 and not done:
                done = True
                reward = -steps
            memory.append((observation, action, reward))
            observation = observation_

        #append the terminal state
        memory.append((observation, action, reward))

        G = 0
        last = True # start at t = T - 1
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:
                statesActionsReturns.append((state,action,G))
            G = GAMMA*G + reward
        statesActionsReturns.reverse()

        statesAndActions = []
        for state, action, G in statesActionsReturns:
            if (state, action) not in statesAndActions:
                pairsVisited[(state,action)] += 1
                returns[(state,action)] += (1 / pairsVisited[(state,action)])*(G-returns[(state,action)])
                Q[(state,action)] = returns[(state,action)]
                statesAndActions.append((state,action))
                values = np.array([Q[(state,a)] for a in grid.possibleActions])
                best = np.random.choice(np.where(values==values.max())[0])
                rand = np.random.random()
                if rand < 1 - EPS:
                    policy[state] = grid.possibleActions[best]
                else:
                    policy[state] = np.random.choice(grid.possibleActions)

    printQ(Q, grid)
    printPolicy(policy,grid)

In [44]:
MC_without_exploring_starts8x8()

starting episode 0
starting episode 100000
starting episode 200000
starting episode 300000
starting episode 400000
starting episode 500000
starting episode 600000
starting episode 700000
starting episode 800000
starting episode 900000
[-75.99996, -75.99996, -76.0, -75.99999]	[-74.99983, -74.9999, -74.99983, -74.99983]	[-74.0, -73.99976, -74.0, -74.0]	[-72.6184, -72.57633, -72.47348, -72.51166]	[-70.60079, -70.64729, -70.725, -70.66357]	[-69.72857, -69.58519, -69.85417, -69.56522]	[-68.72222, -68.6129, -68.63158, -68.42857]	[-67.22222, -67.28571, -67.33333, -67.36364]	

[-67.0, -69.0, -66.5, -69.0]	[-71.98656, -71.98995, -71.99001, -71.98976]	[-73.0, -73.0, -72.99965, -73.0]	[-71.83545, -71.90227, -71.83784, -71.89681]	[-68.0, -68.0, -62.0, -68.0]	[-65.0, -64.25, -66.0, -65.0]	[-67.5, -67.0, -67.66667, -68.0]	[-66.66667, -65.625, -67.0, -66.0]	

[-68.0, -68.0, -68.0, -67.66667]	[-70.82143, -70.83051, -70.81159, -70.80029]	[-72.0, -72.0, -72.0, -72.0]	[-71.0, -71.0, -71.0, -70.94704]	[0,

In [49]:
def MC_without_exploring_starts10x10():
    grid = WindyGrid(10,10, wind=[2, 1, 2, 0, 3, 2, 2, 1, 1, 0])
    GAMMA = 0.9
    EPS = 0.4

    Q = {}
    returns = {}
    pairsVisited = {}
    for state in grid.stateSpacePlus:
        for action in grid.actionSpace.keys():
            Q[(state, action)] = 0
            returns[(state,action)] = 0
            pairsVisited[(state,action)] = 0

    policy = {}
    for state in grid.stateSpace:
        policy[state] = grid.possibleActions

    for i in range(1000000):
        statesActionsReturns = []
        if i % 100000 == 0:
            print('starting episode', i)
        observation, done = grid.reset()
        memory = []
        steps = 0
        while not done:
            if len(policy[observation]) > 1:
                action = np.random.choice(policy[observation])
            else:
                action = policy[observation]
            observation_, reward, done, info = grid.step(action)
            steps += 1
            if steps > 25 and not done:
                done = True
                reward = -steps
            memory.append((observation, action, reward))
            observation = observation_

        #append the terminal state
        memory.append((observation, action, reward))

        G = 0
        last = True # start at t = T - 1
        for state, action, reward in reversed(memory):
            if last:
                last = False
            else:
                statesActionsReturns.append((state,action,G))
            G = GAMMA*G + reward
        statesActionsReturns.reverse()

        statesAndActions = []
        for state, action, G in statesActionsReturns:
            if (state, action) not in statesAndActions:
                pairsVisited[(state,action)] += 1
                returns[(state,action)] += (1 / pairsVisited[(state,action)])*(G-returns[(state,action)])
                Q[(state,action)] = returns[(state,action)]
                statesAndActions.append((state,action))
                values = np.array([Q[(state,a)] for a in grid.possibleActions])
                best = np.random.choice(np.where(values==values.max())[0])
                rand = np.random.random()
                if rand < 1 - EPS:
                    policy[state] = grid.possibleActions[best]
                else:
                    policy[state] = np.random.choice(grid.possibleActions)

    printQ(Q, grid)
    printPolicy(policy,grid)

In [50]:
MC_without_exploring_starts10x10()

starting episode 0
starting episode 100000
starting episode 200000
starting episode 300000
starting episode 400000
starting episode 500000
starting episode 600000
starting episode 700000
starting episode 800000
starting episode 900000
[-13.1428, -13.14296, -13.14293, -13.1428]	[-13.492, -13.49422, -13.49301, -13.49204]	[-13.88403, -13.88006, -13.88095, -13.88214]	[-14.31111, -14.31111, -14.33348, -14.31111]	[-14.81366, -15.13659, -15.44825, -14.82317]	[-15.47677, -15.47841, -15.63447, -15.82713]	[-19.13893, -18.40202, -19.82583, -20.01495]	[0, -33.26531, -21.12772, -41.914]	[-26.0, 0, 0, -45.46]	[0, 0, -49.4, 0]	

[-25.26437, 0, -23.73793, -26.96041]	[0, 0, 0, 0]	[-16.78235, -16.10446, -17.056, -15.42144]	[-14.79012, -14.79012, -14.79012, -14.85619]	[-15.41535, -17.25748, -16.88655, -16.06058]	[0, 0, 0, -26.96041]	[0, 0, 0, 0]	[0, 0, 0, 0]	[0, 0, 0, 0]	[0, 0, 0, 0]	

[0, 0, 0, 0]	[0, 0, -22.36414, 0]	[-15.91373, -15.91373, -19.59596, -16.60731]	[-15.32236, -15.32236, -15.32236, -15.322

## Off-Policy Monte Carlo prediction

In [52]:
def MC_off_policy_prediction():
    grid = WindyGrid(6,6, wind=[0,0,1,2,1,0])
    GAMMA = 0.9

    Q = {}
    C = {}
    for state in grid.stateSpacePlus:
        for action in grid.possibleActions:
            Q[(state,action)] = 0
            C[(state,action)] = 0

    targetPolicy = {}
    for state in grid.stateSpace:
        targetPolicy[state] = np.random.choice(grid.possibleActions)

    for i in range(1000000):
        if i % 100000 == 0:
            print(i)
        behaviorPolicy = {}
        for state in grid.stateSpace:
            behaviorPolicy[state] = grid.possibleActions
        memory = []
        observation, done = grid.reset()
        steps = 0
        while not done:
            action = np.random.choice(behaviorPolicy[observation])
            observation_, reward, done, info = grid.step(action)
            steps += 1
            if steps > 25:
                done = True
                reward = -steps
            memory.append((observation, action, reward))
            observation = observation_
        memory.append((observation, action, reward))

        G = 0
        W = 1
        last = True
        for (state, action, reward) in reversed(memory):
            if last:
                last = False
            else:
                C[state,action] += W
                Q[state,action] += (W / C[state,action])*(G-Q[state,action])
                prob = 1 if action in targetPolicy[state] else 0
                W *= prob/(1/len(behaviorPolicy[state]))
                if W == 0:
                    break
            G = GAMMA*G + reward
    printQ(Q, grid)
    printPolicy(targetPolicy,grid)

In [53]:
MC_off_policy_prediction()

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
[-38.05781, -37.52833, -37.00381, -35.15851]	[-37.84606, -35.37806, -36.54147, -36.96263]	[-37.44392, -38.10604, -38.34291, -37.73372]	[-32.34379, -37.74501, -37.72947, -36.41051]	[-38.16075, -38.39015, -36.90478, -34.08695]	[-38.9, -39.0271, -35.90988, -33.43134]	

[-34.88785, -37.37171, -37.39188, -33.55938]	[-35.95484, -38.7313, -36.69029, -35.99513]	[-32.48952, -32.20699, -35.7896, -39.97103]	[-39.78629, -38.95528, -39.79015, -39.44922]	[-40.17366, -39.81162, -40.03659, -39.30259]	[-39.88603, -38.42122, -37.80349, -39.27991]	

[-33.36636, -37.36397, -39.21693, -33.76473]	[-35.59385, -38.92496, -37.08644, -37.31776]	[-35.40367, -39.53461, -39.4144, -38.73432]	[-39.2927, -40.31266, -36.21248, -40.33113]	[-40.38411, -40.01968, -39.35724, -39.29753]	[-39.19433, -40.01608, -39.79852, -38.17689]	

[-28.74084, -38.57146, -38.17726, -36.74418]	[-37.9957, -38.934, -36.05406, -38.80411]	[-34.46779, -38.31229, -30.65532, -39.520

## Del 4
- Använd *off-policy prediction* Monte Carlo Metoden

1. Öka vindstyrkan med en enhet.
    - Hur ändras slutvärdesfunktionen?
        Svar: Medeltalet av negative värden steg lite, men policyn klarar sig rätt väl och relativt oförändrad

2. Hur ändras policyn om man ändra gamma till:
    - 𝛾=0.5 Svar: Policyn klarar sig väldigt bra och rör sig emellan -8 - -30
    - 𝛾=0,9 Svar: Policyns värden ligger mellan -36 och -42
    - 𝛾=0,95 Svar: Policyn ligger i medelvärde kring -42 - -44


3. Testa rutnätsvärlden i storlekarna:
    - 8x8
        - Ändra på vinden, vad händer med policyn?
            Svar: värdena rör sig mellan -52 och -54, så svårare blev det nog för agenten
        - Prova med 𝛾=0,9, vad händer med policyn?
            Svar: gamma 0,9 klarar sig bättre än 1
    - 10x10
        - Ändra på vinden, vad händer med policyn?
            Svar: Policyn klarar sig rätt bra, trots att fältet är större och vinden är ökad

In [60]:
def MC_off_policy_predictionWindy():
    grid = WindyGrid(6,6, wind=[2,0,2,3,2,0])
    GAMMA = 0.9

    Q = {}
    C = {}
    for state in grid.stateSpacePlus:
        for action in grid.possibleActions:
            Q[(state,action)] = 0
            C[(state,action)] = 0

    targetPolicy = {}
    for state in grid.stateSpace:
        targetPolicy[state] = np.random.choice(grid.possibleActions)

    for i in range(1000000):
        if i % 100000 == 0:
            print(i)
        behaviorPolicy = {}
        for state in grid.stateSpace:
            behaviorPolicy[state] = grid.possibleActions
        memory = []
        observation, done = grid.reset()
        steps = 0
        while not done:
            action = np.random.choice(behaviorPolicy[observation])
            observation_, reward, done, info = grid.step(action)
            steps += 1
            if steps > 25:
                done = True
                reward = -steps
            memory.append((observation, action, reward))
            observation = observation_
        memory.append((observation, action, reward))

        G = 0
        W = 1
        last = True
        for (state, action, reward) in reversed(memory):
            if last:
                last = False
            else:
                C[state,action] += W
                Q[state,action] += (W / C[state,action])*(G-Q[state,action])
                prob = 1 if action in targetPolicy[state] else 0
                W *= prob/(1/len(behaviorPolicy[state]))
                if W == 0:
                    break
            G = GAMMA*G + reward
    printQ(Q, grid)
    printPolicy(targetPolicy,grid)

In [ ]:
MC_off_policy_predictionWindy()

0
100000


In [4]:
def MC_off_policy_predictionY():
    grid = WindyGrid(6,6, wind=[2,0,2,3,2,0])
    GAMMA = 0.95

    Q = {}
    C = {}
    for state in grid.stateSpacePlus:
        for action in grid.possibleActions:
            Q[(state,action)] = 0
            C[(state,action)] = 0

    targetPolicy = {}
    for state in grid.stateSpace:
        targetPolicy[state] = np.random.choice(grid.possibleActions)

    for i in range(1000000):
        if i % 100000 == 0:
            print(i)
        behaviorPolicy = {}
        for state in grid.stateSpace:
            behaviorPolicy[state] = grid.possibleActions
        memory = []
        observation, done = grid.reset()
        steps = 0
        while not done:
            action = np.random.choice(behaviorPolicy[observation])
            observation_, reward, done, info = grid.step(action)
            steps += 1
            if steps > 25:
                done = True
                reward = -steps
            memory.append((observation, action, reward))
            observation = observation_
        memory.append((observation, action, reward))

        G = 0
        W = 1
        last = True
        for (state, action, reward) in reversed(memory):
            if last:
                last = False
            else:
                C[state,action] += W
                Q[state,action] += (W / C[state,action])*(G-Q[state,action])
                prob = 1 if action in targetPolicy[state] else 0
                W *= prob/(1/len(behaviorPolicy[state]))
                if W == 0:
                    break
            G = GAMMA*G + reward
    printQ(Q, grid)
    printPolicy(targetPolicy,grid)

In [5]:
MC_off_policy_predictionY()

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
[-42.57625, -43.57025, -43.14464, -43.63798]	[-44.22777, -43.54978, -43.4369, -43.8944]	[-43.91264, -42.28521, -43.59771, -43.47755]	[-43.9393, -43.80614, -43.99276, -43.97045]	[-43.96838, -43.65026, -43.87809, -43.73913]	[-43.90589, -43.65859, -43.83027, -43.68605]	

[-43.61045, -43.94145, -44.20668, -40.92329]	[-43.81334, -42.76214, -39.7199, -44.04583]	[-44.23892, -44.20638, -43.66344, -44.19882]	[-43.75631, -43.48755, -43.24941, -43.81391]	[-43.92868, -42.77372, -43.69055, -43.65318]	[-43.75598, -43.84408, -43.65722, -43.64151]	

[-43.92818, -43.7478, -43.55722, -44.38002]	[-44.11981, -44.15267, -43.28909, -43.90108]	[-44.49204, -44.12212, -42.954, -43.45948]	[-41.14323, -38.35, -41.49804, -42.5975]	[-43.71235, -43.97342, -43.2782, -42.72402]	[-44.18386, -43.48263, -43.56544, -43.82198]	

[-44.27784, -44.8645, -43.76423, -44.17511]	[-44.56681, -44.464, -44.45598, -43.90697]	[-43.53626, -43.81704, -44.16053, -43.69391]

In [9]:
def MC_off_policy_prediction8x8():
    grid = WindyGrid(8,8, wind=[1,3,0,2,2,1,1,0])
    GAMMA = 0.9

    Q = {}
    C = {}
    for state in grid.stateSpacePlus:
        for action in grid.possibleActions:
            Q[(state,action)] = 0
            C[(state,action)] = 0

    targetPolicy = {}
    for state in grid.stateSpace:
        targetPolicy[state] = np.random.choice(grid.possibleActions)

    for i in range(1000000):
        if i % 100000 == 0:
            print(i)
        behaviorPolicy = {}
        for state in grid.stateSpace:
            behaviorPolicy[state] = grid.possibleActions
        memory = []
        observation, done = grid.reset()
        steps = 0
        while not done:
            action = np.random.choice(behaviorPolicy[observation])
            observation_, reward, done, info = grid.step(action)
            steps += 1
            if steps > 25:
                done = True
                reward = -steps
            memory.append((observation, action, reward))
            observation = observation_
        memory.append((observation, action, reward))

        G = 0
        W = 1
        last = True
        for (state, action, reward) in reversed(memory):
            if last:
                last = False
            else:
                C[state,action] += W
                Q[state,action] += (W / C[state,action])*(G-Q[state,action])
                prob = 1 if action in targetPolicy[state] else 0
                W *= prob/(1/len(behaviorPolicy[state]))
                if W == 0:
                    break
            G = GAMMA*G + reward
    printQ(Q, grid)
    printPolicy(targetPolicy,grid)

In [10]:
MC_off_policy_prediction8x8()

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
[-36.40153, -37.50964, -36.39348, -28.9872]	[-37.92264, -35.64033, -35.63362, -33.35021]	[-36.41845, -36.07152, -36.12156, -35.77309]	[-37.36407, -37.19363, -37.08183, -30.19848]	[-38.32023, -36.64848, -34.20683, -37.52381]	[-38.96099, -38.76363, -38.68576, -37.51531]	[-38.91062, -38.52913, -37.45711, -38.99749]	[-39.00536, -39.27524, -39.48146, -39.0092]	

[-40.09905, -40.00812, -40.81561, -38.13333]	[-29.30774, -33.53326, -38.745, -39.5257]	[-37.23952, -39.04061, -37.16487, -37.23678]	[-38.73169, -39.10804, -36.8967, -38.86727]	[-40.24824, -41.32133, -40.05687, -41.78643]	[-37.51248, -36.99053, -39.56139, -38.69565]	[-38.94267, -36.19676, -40.54877, -38.88079]	[-39.43835, -39.8209, -39.65912, -40.01672]	

[-42.13889, -38.76364, -45.59314, -43.01818]	[-40.26338, -40.6778, -40.56389, -36.83279]	[-39.9011, -38.92728, -39.948, -39.77168]	[-40.49588, -40.61973, -39.7842, -40.62486]	[-32.68571, -42.99325, -42.00432, -36.124]	

In [12]:
def MC_off_policy_prediction10x10():
    grid = WindyGrid(10,10, wind=[1, 1,3,0,2,2,1,1,0, 1])
    GAMMA = 0.9

    Q = {}
    C = {}
    for state in grid.stateSpacePlus:
        for action in grid.possibleActions:
            Q[(state,action)] = 0
            C[(state,action)] = 0

    targetPolicy = {}
    for state in grid.stateSpace:
        targetPolicy[state] = np.random.choice(grid.possibleActions)

    for i in range(1000000):
        if i % 100000 == 0:
            print(i)
        behaviorPolicy = {}
        for state in grid.stateSpace:
            behaviorPolicy[state] = grid.possibleActions
        memory = []
        observation, done = grid.reset()
        steps = 0
        while not done:
            action = np.random.choice(behaviorPolicy[observation])
            observation_, reward, done, info = grid.step(action)
            steps += 1
            if steps > 25:
                done = True
                reward = -steps
            memory.append((observation, action, reward))
            observation = observation_
        memory.append((observation, action, reward))

        G = 0
        W = 1
        last = True
        for (state, action, reward) in reversed(memory):
            if last:
                last = False
            else:
                C[state,action] += W
                Q[state,action] += (W / C[state,action])*(G-Q[state,action])
                prob = 1 if action in targetPolicy[state] else 0
                W *= prob/(1/len(behaviorPolicy[state]))
                if W == 0:
                    break
            G = GAMMA*G + reward
    printQ(Q, grid)
    printPolicy(targetPolicy,grid)

In [13]:
MC_off_policy_prediction10x10()

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
[-35.99384, -37.62612, -36.80307, -38.03425]	[-35.74897, -37.84433, -38.1226, -33.35896]	[-36.17176, -32.65382, -36.12827, -35.28551]	[-36.89722, -38.11074, -37.6985, -39.0713]	[-39.04348, -38.35821, -35.72594, -38.61525]	[-35.05473, -38.00016, -34.98939, -37.83342]	[-35.77162, -38.67482, -39.4914, -38.37358]	[-38.10008, -39.34053, -38.77216, -38.72812]	[-37.7216, -30.40062, -39.81335, -39.71349]	[-39.61346, -38.33476, -39.87662, -39.18638]	

[-26.0, -44.72, -26.0, -45.46]	[-39.06323, -40.09905, -38.78359, -33.2]	[-39.88026, -37.81541, -39.54251, -37.70885]	[-36.82179, -36.71122, -39.12847, -39.29959]	[-39.52575, -38.17523, -39.85703, -39.99454]	[-39.71937, -36.7989, -41.60576, -37.38569]	[-26.0, -41.43176, -49.4, -45.46]	[-36.97306, -39.57496, -40.24444, -40.00013]	[-32.00314, -5.17355, -39.96474, -38.94218]	[-39.61591, -40.13213, -37.65855, -39.6817]	

[0, 0, 0, -26.0]	[-41.44578, -37.7, -26.0, -26.0]	[-40.37126, -39.94